In [ ]:
!pip install catboost

In [ ]:
!pip install optuna

In [ ]:
!pip install pytorch-tabnet

In [ ]:
!pip install xgboost

In [ ]:
import os
import sys
import platform
import random
import math

import pandas as pd
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.metrics import Metric

import sklearn 
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score 
from sklearn.preprocessing import LabelEncoder
import torch
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import Pool,CatBoostClassifier


import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def seed_all(seed: int = 1930):

    print("Using Seed Number {}".format(seed))

    os.environ["PYTHONHASHSEED"] = str(
        seed)  # set PYTHONHASHSEED env var at fixed value
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)
    # torch.cuda.manual_seed(seed)  # pytorch (both CPU and CUDA)
    np.random.seed(seed)  # for numpy pseudo-random generator
    random.seed(
        seed)  # set fixed value for python built-in pseudo-random generator
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = False
    # torch.backends.cudnn.enabled = False


def seed_worker(_worker_id):
    # worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
seed_all(seed=1994)


In [ ]:
df_train = pd.read_csv('./drive/MyDrive/Dacon/jobcare/data/train_pp.csv')
df_test = pd.read_csv('./drive/MyDrive/Dacon/jobcare/data/test_pp.csv')

In [ ]:
train_X = df_train.drop(['target'], axis=1)
train_y = df_train.target

test_X = df_test

### 명세서 기준 순서형 변수와 명목형 변수를 구분

In [ ]:
# 수치형
numeric_var = ['matching_num']

# 명목형 변수
var = list(train_X.columns)

cate_var = [x for x in var if x not in numeric_var]
cate_var

In [ ]:
# class MultiColumnLabelEncoder:
#     def __init__(self,columns = None):
#         self.columns = columns # array of column names to encode

#     def fit(self,X,y=None):
#         return self # not relevant here

#     def transform(self,X):
#         '''
#         Transforms columns of X specified in self.columns using
#         LabelEncoder(). If no columns specified, transforms all
#         columns in X.
#         '''
#         output = X.copy()
#         if self.columns is not None:
#             for col in self.columns:
#                 output[col] = LabelEncoder().fit_transform(output[col])
#         else:
#             for colname,col in output.iteritems():
#                 output[colname] = LabelEncoder().fit_transform(col)
#         return output

#     def fit_transform(self,X,y=None):
#         return self.fit(X,y).transform(X)


In [ ]:
# le=MultiColumnLabelEncoder(columns=var)
# le.fit(train_X)
# train_X = le.transform(train_X)
# test_X = le.transform(test_X)

In [ ]:
cardinality = train_X.apply(pd.Series.nunique)
cat_idxs = [ i for i, f in enumerate(var) if f in cate_var]
cat_dims = [cardinality[i] for i in cate_var]

# LGBM K-Fold

In [ ]:
for c in cate_var:
    train_X[c] = train_X[c].astype('category')
    test_X[c] = test_X[c].astype('category')

* {'bagging_fraction': 0.918642726084944,
  'feature_fraction': 0.509220798163292,
  'lambda_l1': 6.755458177849535,
  'lambda_l2': 0.009582194283353274,
  'num_leaves': 24}

In [ ]:
n_splits = 30
LGBM_test = np.zeros(test_X.shape[0]) # test prediction
LGBM_train = np.zeros(train_X.shape[0])

skf = StratifiedKFold(n_splits = n_splits, random_state = 42, shuffle = True)

for i, (t_idx, v_idx) in enumerate(skf.split(train_X, train_y)):
    print(f"============================{i+1}============================")

    X_train, y_train = train_X.iloc[t_idx], train_y.iloc[t_idx].values
    X_valid, y_valid = train_X.iloc[v_idx], train_y.iloc[v_idx].values

    LGBM = LGBMClassifier(random_state = 42, n_estimators = 3000, 
                          categorical_feature = cate_var, eval_metric="F1",
                          bagging_fraction=0.918642726084944, feature_fraction=0.509220798163292,
                          lambda_l1=6.755458177849535, lambda_l2=0.009582194283353274,
                          num_leaves=24
                          )
    LGBM.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_valid, y_valid)], early_stopping_rounds = 100, verbose = 100)

    
    LGBM_test += (LGBM.predict_proba(test_X)[:, 1])/n_splits # test set prediction
    LGBM_train[v_idx] += LGBM.predict_proba(X_valid)[:, 1]

    # pred = LGBM.predict_proba(X_valid)[:, 1]
    # pred_ = np.where(pred >= 0.5 , 1, 0)
    # score_ = f1_score(y_valid,pred_)
    # LGBM_scores.append(score_)
    # print(f"threshold 0.5 score: {score_}")

    # pred = np.where(pred >= 0.4 , 1, 0)
    # score = f1_score(y_valid,pred)
    # LGBM_scores_thres.append(score)
    # print(f"threshold 0.4 score: {score}")

In [ ]:
LGBM_train

In [ ]:
threshold = 0.4
train_pred = np.where(LGBM_train >= 0.4 , 1, 0)
train_pred

In [ ]:
train_score = f1_score(train_pred, train_y)
train_score

* 50fold: 0.6894
* 30fold: 0.6893
* 25fold: 0.6886
* 20fold: 0.6886
* 15fold: 0.6883
* 10fold: 0.6873

In [ ]:
# print(f'threshold_0.5: {np.mean(LGBM_scores)}')
# print(f'threshold_0.4: {np.mean(LGBM_scores_thres)}')

# TabNet K-Fold

In [ ]:
class F1_Score(Metric):
    def __init__(self):
        self._name = "f1"
        self._maximize = True

    def __call__(self, y_true, y_score):
        score = f1_score(y_true, (y_score[:, 1]>0.5)*1)
        return score

In [ ]:
n_splits = 10
threshold = 0.4
Tab_test = np.zeros(test_X.shape[0]) # test prediction
Tab_train = np.zeros(train_X.shape[0])

skf = StratifiedKFold(n_splits = n_splits, random_state = 42, shuffle = True)

for i, (t_idx, v_idx) in enumerate(skf.split(train_X, train_y)):
    print(f"============================{i+1}============================")

    X_train, y_train = train_X.iloc[t_idx].values, train_y.iloc[t_idx].values
    X_valid, y_valid = train_X.iloc[v_idx].values, train_y.iloc[v_idx].values
    Tab = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=3,
                       optimizer_fn=torch.optim.AdamW, # Any optimizer works here
                       mask_type='entmax', # "sparsemax",
                      )
    Tab.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_name=['train', 'val'],
        eval_metric=['logloss','f1'],
        max_epochs=100, patience=5,
        batch_size=1024,
        virtual_batch_size=256,
        num_workers=1,
        drop_last=False,
    )

    Tab_test += (Tab.predict_proba(test_X.values)[:,1])/n_splits     # test set prediction
    Tab_train[v_idx] += Tab.predict_proba(X_valid)[:, 1]
    
    # pred = Tab.predict_proba(X_valid)[:,1]
    # pred_th = np.where(pred >= threshold , 1, 0)
    # score = f1_score(y_valid,pred_th)
    # Tab_scores_thres.append(score)

    # pred_ = np.where(pred >= 0.5 , 1, 0)
    # score = f1_score(y_valid,pred_)
    # Tab_scores.append(score)

* 10fold: 0.6610, 0.6884
* 15fold: 0.6578, 0.6872
* 20fold: 0.6583, 0.6880

In [ ]:
Tab_train

In [ ]:
threshold = 0.4
train_pred = np.where(Tab_train >= 0.4 , 1, 0)
train_pred

In [ ]:
train_score = f1_score(train_pred, train_y)
train_score

In [ ]:
print(f'threshold_0.5: {np.mean(Tab_scores)}')
print(f'threshold_0.4: {np.mean(Tab_scores_thres)}')

## stacking ensemble을 위한 column 추가

In [ ]:
len(train_X)

In [ ]:
LGBM_train

In [ ]:
len(Tab_train)

In [ ]:
train_X['LGBM_prob'] = LGBM_train
# train_X['Tab_prob'] = Tab_train

In [ ]:
test_X['LGBM_prob'] = LGBM_test
# test_X['Tab_prob'] = Tab_test

In [ ]:
train_X.to_csv('./drive/MyDrive/Dacon/jobcare/data/train_stacking.csv', index=False)
test_X.to_csv('./drive/MyDrive/Dacon/jobcare/data/test_stacking.csv', index=False)

In [ ]:
train_X = pd.read_csv('./drive/MyDrive/Dacon/jobcare/data/train_stacking.csv')
test_X = pd.read_csv('./drive/MyDrive/Dacon/jobcare/data/test_stacking.csv')

# catboost K-Fold

In [ ]:
# cat_features = train_X.columns[train_X.nunique() > 2].tolist()

In [ ]:
{'objective': 'CrossEntropy', 'depth': 12, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 1.4079670122251873}

In [ ]:
n_splits = 10
iterations = 3000
early_stopping_rounds = 300
CBC_test = np.zeros(test_X.shape[0])
CBC_train = np.zeros(train_X.shape[0])
threshold = 0.4

skf = StratifiedKFold(n_splits = n_splits, random_state = 42, shuffle = True)

for i, (t_idx, v_idx) in enumerate(skf.split(train_X, train_y)):
    print(f"============================{i+1}============================")

    X_train, y_train = train_X.iloc[t_idx], train_y.iloc[t_idx].values
    X_valid, y_valid = train_X.iloc[v_idx], train_y.iloc[v_idx].values
    CBC = CatBoostClassifier(iterations=iterations,random_state=42,task_type="GPU",
                             eval_metric="F1",cat_features=cat_idxs,one_hot_max_size=4,
                             bagging_temperature=1.4079670122251873, boosting_type="Plain",
                             bootstrap_type="Bayesian", depth=12, objective="CrossEntropy"
                             )
    CBC.fit(X_train, y_train, 
            eval_set=[(X_valid, y_valid)],
            early_stopping_rounds=early_stopping_rounds,
            verbose = 100
        )    
    
    CBC_test += (CBC.predict_proba(test_X)[:, 1])/n_splits # test set prediction
    CBC_train[v_idx] = CBC.predict_proba(X_valid)[:, 1]

    # pred = np.where(pred >= threshold , 1, 0)
    # score = f1_score(y_valid,pred)
    # CBC_scores_thres.append(score)

    # CBC_scores.append(CBC.get_best_score()["validation"]["F1"])

* 25fold: 0.6849, 0.7119
* 15fold: 0.6832, 0.7121
* 10fold: 0.6839, 0.7123 \***
* 5fold: 0.6709, 0.7096


In [ ]:
CBC_train

In [ ]:
threshold = 0.4
train_pred = np.where(CBC_train >= 0.4 , 1, 0)
train_pred

In [ ]:
train_score = f1_score(train_pred, train_y)
train_score

# XGB K-fold

In [ ]:
n_splits = 50
XGB_scores = []
XGB_scores_thres = []
threshold = 0.4
XGB_pred = np.zeros(test_X.shape[0]) # test prediction

skf = StratifiedKFold(n_splits = n_splits, random_state = 42, shuffle = True)

for i, (t_idx, v_idx) in enumerate(skf.split(train_X, train_y)):
    print(f"============================{i+1}============================")

    X_train, y_train = train_X.iloc[t_idx], train_y.iloc[t_idx].values
    X_valid, y_valid = train_X.iloc[v_idx], train_y.iloc[v_idx].values
    XGB = XGBClassifier(random_state = 42, n_estimators = 3000, eval_metric="F1",
                        tree_method="gpu_hist", enable_categorical=True, use_label_encoder=False)
    

    XGB.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_valid, y_valid)], early_stopping_rounds = 100, verbose = 100)

    
    XGB_pred += (XGB.predict_proba(test_X)[:, 1])/n_splits # test set prediction
    pred = XGB.predict_proba(X_valid)[:, 1]

    pred_ = np.where(pred >= 0.5 , 1, 0)
    score_ = f1_score(y_valid,pred_)
    XGB_scores.append(score_)
    print(f"threshold 0.5 score: {score_}")

    pred = np.where(pred >= 0.4 , 1, 0)
    score = f1_score(y_valid,pred)
    XGB_scores_thres.append(score)
    print(f"threshold 0.4 score: {score}")

# Optuna

In [ ]:
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y)

* CatBoost

In [ ]:
def CBC_objective(trial):
    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ), "iterations": 3000, "random_state": 42, "task_type": "GPU", "eval_metric": "F1", 
        "cat_features": cat_idxs, "one_hot_max_size": 4
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    CBC = CatBoostClassifier(**param)

    CBC.fit(train_X, train_y, 
            eval_set=[(valid_X, valid_y)],
            early_stopping_rounds=200,
            verbose = 100
    )
    prob = CBC.predict_proba(valid_X)[:, 1]
    pred = np.where(prob >= 0.4 , 1, 0)
    score = f1_score(pred, valid_y)

    return score

In [ ]:
CBC_study = optuna.create_study(direction="maximize")
CBC_study.optimize(CBC_objective, n_trials=1000)

In [ ]:
CBC_study.best_trial.value, CBC_study.best_trial.params

(0.7083990423506532,
 {'objective': 'CrossEntropy', 'depth': 12, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 1.4079670122251873})

In [ ]:























af
























In [ ]:
CBC_params = CBC_study.best_params
print(CBC_params)
# Optimized model
# CBC = CatBoostClassifier(**CBC_params)

* TabNet

In [ ]:
train_X, train_y, valid_X, valid_y = train_X.values, train_y.values, valid_X.values, valid_y.values

In [ ]:
n_splits = 10
threshold = 0.4
Tab_test = np.zeros(test_X.shape[0]) # test prediction
Tab_train = np.zeros(train_X.shape[0])

skf = StratifiedKFold(n_splits = n_splits, random_state = 42, shuffle = True)

for i, (t_idx, v_idx) in enumerate(skf.split(train_X, train_y)):
    print(f"============================{i+1}============================")

    X_train, y_train = train_X.iloc[t_idx].values, train_y.iloc[t_idx].values
    X_valid, y_valid = train_X.iloc[v_idx].values, train_y.iloc[v_idx].values
    Tab = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=3,
                       optimizer_fn=torch.optim.AdamW, # Any optimizer works here
                       mask_type='entmax', # "sparsemax",
                      )
    Tab.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_name=['train', 'val'],
        eval_metric=['logloss','f1'],
        max_epochs=100, patience=5,
        batch_size=1024,
        virtual_batch_size=256,
        num_workers=1,
        drop_last=False,
    )

    Tab_test += (Tab.predict_proba(test_X.values)[:,1])/n_splits     # test set prediction
    Tab_train[v_idx] += Tab.predict_proba(X_valid)[:, 1]
    
    # pred = Tab.predict_proba(X_valid)[:,1]
    # pred_th = np.where(pred >= threshold , 1, 0)
    # score = f1_score(y_valid,pred_th)
    # Tab_scores_thres.append(score)

    # pred_ = np.where(pred >= 0.5 , 1, 0)
    # score = f1_score(y_valid,pred_)
    # Tab_scores.append(score)

In [ ]:
def Tab_objective(trial):
    mask_type = trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    n_da = trial.suggest_int("n_da", 56, 64, step=4)
    n_steps = trial.suggest_int("n_steps", 1, 3, step=1)
    gamma = trial.suggest_float("gamma", 1., 1.4, step=0.2)
    n_shared = trial.suggest_int("n_shared", 1, 3)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    tabnet_params = dict(n_d=n_da, n_a=n_da, n_steps=n_steps, gamma=gamma,
                     lambda_sparse=lambda_sparse,
                     mask_type=mask_type, n_shared=n_shared,
                     verbose=0, cat_dims=cat_dims, cat_idxs=cat_idxs
                     )
    # param = {
    #     'mask_type': trial.suggest_categorical("mask_type", ["entmax", "sparsemax"]),
    #     "n_da": trial.suggest_int("n_da", 56, 64, step=4),
    #     'n_steps': trial.suggest_int("n_steps", 1, 3, step=1),
    #     "gamma": trial.suggest_float("gamma", 1., 1.4, step=0.2),
    #     'n_shared': trial.suggest_int("n_shared", 1, 3),
    #     'lambda_sparse': trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True),
    #     'cat_dims': cat_idxs, "cat_dims": cat_dims, "cat_emb_dim": 5, 
    # }

    

    Tab = TabNetClassifier(**tabnet_params)

    Tab.fit(
        X_train=train_X, y_train=train_y,
        eval_set=[(train_X, train_y), (valid_X, valid_y)],
        eval_name=['train', 'val'],
        eval_metric=['logloss'],
        max_epochs=100,
        batch_size=1024,
        virtual_batch_size=256,
        num_workers=1,
        drop_last=False,
    )

    pred = Tab.predict_proba(X_valid)[:, 1]
    pred_ = np.where(pred >= 0.5 , 1, 0)
    score_ = f1_score(y_valid,pred_)
    print(f"threshold 0.5 score: {score_}")

    return score_

In [ ]:
Tab_study = optuna.create_study(direction="maximize")
Tab_study.optimize(Tab_objective, n_trials=2)

* LGBM

In [ ]:
for c in cate_var:
    train_X[c] = train_X[c].astype('category')
    valid_X[c] = valid_X[c].astype('category')
    test_X[c] = test_X[c].astype('category')

In [ ]:
def LGBM_objective(trial):
    param = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'objective': 'regression',
        'learning_rate': 0.1,
        "boosting": "gbdt",
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        "bagging_freq": 5,
        "bagging_fraction": trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        "feature_fraction": trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        "verbosity": 100, "random_state": 42, "n_estimators":  3000, "categorical_feature": cate_var,
        "eval_metric": "F1"
    }


    LGBM = LGBMClassifier(**param)
    LGBM.fit(train_X, train_y, eval_set = [(train_X, train_y), (valid_X, valid_y)], early_stopping_rounds = 100, verbose = 100)

    pred = LGBM.predict_proba(valid_X)[:, 1]
    pred_ = np.where(pred >= 0.5 , 1, 0)
    score_ = f1_score(valid_y,pred_)
    print(f"threshold 0.5 score: {score_}")


    return score_

In [ ]:
LGBM_study = optuna.create_study(direction="maximize")
LGBM_study.optimize(LGBM_objective, n_trials=1000)

* LGBM 최적 value & parameter: (0.6480186032209709,
 {'bagging_fraction': 0.918642726084944,
  'feature_fraction': 0.509220798163292,
  'lambda_l1': 6.755458177849535,
  'lambda_l2': 0.009582194283353274,
  'num_leaves': 24})

In [ ]:
LGBM_study.best_trial.value, LGBM_study.best_trial.params

# Ensemble

In [ ]:
CBC_pred

In [ ]:
Tab_pred

In [ ]:
LGBM_pred

In [ ]:
pred = [1 if value >= 0.4 else 0 for value in CBC_pred ]


# submission

In [ ]:
submission = pd.read_csv('./drive/MyDrive/Dacon/jobcare/data/sample_submission.csv')
submission['target'] = pred
submission.to_csv('./drive/MyDrive/Dacon/jobcare/submission/cbc+feature_add.csv',index=False)